<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/llm/wizardlm13b_v1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [ ]:
!pip install auto-gptq

Load model

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

model_name_or_path = "TheBloke/WizardLM-13B-V1.2-GPTQ"
model_basename = "gptq_model-4bit-128g"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=False,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

Inference

In [ ]:
prompt = "日本語でAIに関するプレゼンテーションを書いてほしい"
prompt_template=f'''A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

USER: {prompt}
ASSISTANT:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0], skip_special_tokens=True)[len(prompt_template):])

In [ ]:
prompt_template = """You act as a prompt generator. Please do the classification task like the following examples.

Prompt: replace chair into sofa at the left side
Output: {"part": "left", "source object": ["chair"], "target object": ["sofa"]}

Prompt: change wooden table into white table in the right part
Output: {"part": "right", "source object": ["wooden table"], "target object": ["white table"]}

Prompt: replace blue chair and red sofa into yellow table and green chair at the bottom
Output: {"part": "bottom", "source object": ["blue chair", "red sofa"], "target object": ["yellow table", "green chair"]}

Prompt: replace sofa and shelf into chair and picture at the right side in the room
Output:"""

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0], skip_special_tokens=True)[len(prompt_template):])
